In [18]:
import json
from webui.utils import ObjectNamespace
from webui.chat import Character
import sounddevice as sd
from llama_cpp import Llama, LlamaGrammar

def chat(char,prompt="Continue the story without me"):
    full_response = ""
    for response in char.generate_text(prompt):
        full_response += response
    audio = char.text_to_speech(full_response)
    if audio: sd.play(*audio)            
    char.messages.append({"role": char.user, "content": prompt}) #add user prompt to history
    char.messages.append({
        "role": char.name,
        "content": full_response,
        "audio": audio
    })
    return full_response

def get_function(char, query, threshold=1.):
    results = char.ltm.get_query(query=query,include=["metadatas", "distances"],type="function",threshold=threshold)
    if len(results): # found function
        metadata = results[0]["metadata"]
        return metadata
    else: return None

def get_args(char, arguments, template, prompt, retries=3):
    response = char.LLM(f"""
              <|im_start|>system
              Complete these arguments ({arguments}) using this template ({template}) while following the user's request.<|im_end|>
              <|im_start|>user
              {prompt}<|im_end|>
              """,grammar=grammar,stop=["<|im_start|>","<|im_end|>"])
    try:
        args = json.loads(response["choices"][0]["text"])
    except Exception as e:
        print(e)
        args = get_args(char, arguments, template, prompt, retries-1) if retries>0 else None
    return args        

grammar = LlamaGrammar.from_file("./models/LLM/json.gbnf")

root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number_28 
number_27 ::= [-

from_string grammar:



In [2]:
"""
This is a demo to showcase function calling
"""
character = Character(
    character_file="./models/Characters/Amaryllis.json",
    model_file="./models/LLM/mistral-7b-openorca.Q4_K_M.gguf"
)
character.LLM = Llama(
    character.model_file,
    n_ctx=character.model_data["params"]["n_ctx"],
    n_gpu_layers=character.model_data["params"]["n_gpu_layers"],
    verbose=True
)
character.loaded=True
character

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


loading sound c:\Users\hongb\Documents\repos\RVC-Chat\.cache\tts\edge\en-US-JennyNeural\e1e11efedb8cce4ce31f5883ee9a04af.wav (738816,) 16000
vc_single unused args: {'model_name': 'Mae_v2'}
before remix: shape=(738816,), max=0.7178962826728821, min=-0.7133902311325073, mean=1.0637065315677319e-05 sr=16000
after remix: shape=(738816,), max=0.949999988079071, min=-0.9440370798110962, mean=1.4076164006837644e-05, sr=16000
Attempting to load c:\Users\hongb\Documents\repos\RVC-Chat\models\RVC/.index\Mae_v2_40k.index....
loaded index: <faiss.swigfaiss.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x000001F1662A6780> >
f0_method=rmvpe
Returning completed audio...
-------------------
Using index:c:\Users\hongb\Documents\repos\RVC-Chat\models\RVC/.index\Mae_v2_40k.index.


In [20]:
"""
We first define the list of function definitions and put them to a vector database.
The description should be potential user prompts that can trigger the function.

arguments: list of argument names that the function uses
**kwargs: dictionary of [type: data type, description: description of argument, required: whether it is required]
"""
functions = [
    ObjectNamespace(
        description = "Can you please play a song for me?|I would love to hear some music.|Can you choose a song for me?Play a song, please.|Let’s enjoy some music together",
        function = "play_song",
        arguments = ["name","search"],
        name = {"type": "str", "description": "name of the song", "required": True},
        search = {"type": "boolean", "description": "search for the song if it doesn't exist"},
    ),
]

for data in functions:
    character.ltm.add_function(**data)

In [32]:
prompt = "Can you play the song despacito for me?"
metadata = get_function(character,prompt,threshold=1.)
print(f"{metadata=}")
args = get_args(character, metadata['arguments'], metadata['template'], prompt)
args

metadata={'arguments': '["name", "search"]', 'function': 'play_song', 'hnsw:space': 'cosine', 'template': '{"name": {"type": "str", "description": "name of the song", "required": true}, "search": {"type": "boolean", "description": "search for the song if it doesn\'t exist"}}', 'type': 'function'}


Llama.generate: prefix-match hit


{'name': 'Despacito', 'search': True}

In [31]:
def play_song(name,search): # dummy function
    print(f"playing {name}. search song on youtube if it doesn't exist: {search}")
eval(metadata["function"])(**args) # evaluate the song

playing Despacito. search song on youtube if it doesn't exist: True
